In [33]:
#Running on Python version 3.12.1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Import Data from US Bureau of Labor Statistics on Excel

In [34]:
Sheet4 = pd.read_excel('Kansas City Employment Data (Github).xlsx', sheet_name="Unemployment tables", usecols='A:N')
Sheet5 = pd.read_excel('Consumer Price Index for All Urban Consumers.xlsx', sheet_name='BLS Data Series', header=11, usecols='A:M', index_col=0).fillna(0)

Cleaning Data

In [35]:
Unemployment = {}
for row,name in enumerate(Sheet4.iloc[:, 0]):
    if name == 'Row Labels':
        Unemployment[f'{Sheet4.iloc[row-1,0]}'] = Sheet4.iloc[row:row+13]


def clean(dictofdfs, indexcol):
    for table in dictofdfs:
        dictofdfs[table].reset_index(inplace=True, drop=True)
        dictofdfs[table].columns = dictofdfs[table].iloc[0]
        dictofdfs[table] = dictofdfs[table][1:].reset_index(drop=True)
        dictofdfs[table].set_index(dictofdfs[table][indexcol], inplace=True)
        dictofdfs[table].drop(indexcol, axis=1, inplace=True)
        dictofdfs[table].fillna(0, inplace=True)
        for col in dictofdfs[table]:
            dictofdfs[table][col] = dictofdfs[table][col].astype(str).str.replace(r'[^\d.]', '', regex=True).astype(float)
        dictofdfs[table].index.name = 'Year'

        
  
clean(Unemployment, 'Row Labels')

OEWS Occupational Employment and Wage Statistics
Kansas City GDP contribution by industry

In [36]:
OWEScolumns3 = ['AREA_TITLE', 'OCC_CODE', 'OCC_TITLE','O_GROUP','TOT_EMP','LOC_QUOTIENT','A_MEAN', 'H_MEDIAN', 'A_MEDIAN'] #2020-2023
OWEScolumns2 = ['area_title', 'occ_code', 'occ_title', 'o_group', 'tot_emp', 'loc_quotient', 'a_mean', 'h_median', 'a_median'] #2019
OWEScolumns1 = ['AREA_NAME', 'OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'LOC QUOTIENT', 'A_MEAN', 'H_MEDIAN', 'A_MEDIAN'] #2014-2018

files = [f'MSA_M{x}_dl.xlsx' for x in range(2014,2024)]
OWES = {}
for file in files:
    print(file)
    try:
        try:
            OWES[f'{file}'] = pd.read_excel(file, sheet_name=0, index_col='AREA_NAME',header=0, usecols=OWEScolumns1)
        except:
            print('columns2')
            OWES[f'{file}'] = pd.read_excel(file, sheet_name=0, index_col='area_title',header=0, usecols=OWEScolumns2)
    except:
        print('columns3')
        OWES[f'{file}'] = pd.read_excel(file, sheet_name=0, index_col='AREA_TITLE',header=0, usecols=OWEScolumns3)

Clean OWES

In [37]:
for df in OWES:
    OWES[df].index.name = 'AREA_TITLE'
    OWES[df].columns = ['OCC_CODE','OCC_TITLE','O_GROUP','TOT_EMP','LOC_QUOTIENT','A_MEAN', 'H_MEDIAN', 'A_MEDIAN']
for df in OWES:
    OWES[df] = OWES[df].loc['Kansas City, MO-KS']
ALLYearsOWES = pd.DataFrame()
for df in OWES:
    OWES[df]['Year'] = f'{df[5:9]}'
    for row in OWES[df]:
        OWES[df]['Title_Year'] = OWES[df]['OCC_TITLE'] + df[5:9]
OWES['MSA_M2014_dl.xlsx']

OWES national data

In [ ]:
OWEScolumns3 = ['OCC_CODE', 'OCC_TITLE','O_GROUP','TOT_EMP','A_MEAN', 'H_MEDIAN', 'A_MEDIAN'] #2020-2023
OWEScolumns2 = ['occ_code', 'occ_title', 'o_group', 'tot_emp', 'a_mean', 'h_median', 'a_median'] #2019
OWEScolumns1 = ['OCC_CODE', 'OCC_TITLE', 'OCC_GROUP', 'TOT_EMP', 'A_MEAN', 'H_MEDIAN', 'A_MEDIAN'] #2014-2018

files = [f'national_M{x}_dl.xlsx' for x in range(2014,2024)]
OWESN = {}
for file in files:
    print(file)
    try:
        try:
            OWESN[f'{file}'] = pd.read_excel(file, sheet_name=0, index_col='OCC_TITLE',header=0, usecols=OWEScolumns1)
        except:
            print('columns2')
            OWESN[f'{file}'] = pd.read_excel(file, sheet_name=0, index_col='occ_title',header=0, usecols=OWEScolumns2)
    except:
        print('columns3')
        OWESN[f'{file}'] = pd.read_excel(file, sheet_name=0, index_col='OCC_TITLE',header=0, usecols=OWEScolumns3)

for df in OWESN:
    OWESN[df].index.name = 'OCC_TITLE'
    OWESN[df].columns = ['OCC_CODE', 'OCC_GROUP', 'TOT_EMP', 'A_MEAN', 'H_MEDIAN', 'A_MEDIAN']
for df in OWESN:
    OWESN[df]['Year'] = f'{df[10:14]}'




I decided to save my filtered and cleaned data in an MySQL schema for easy access.

MySQL Connection

In [39]:
from urllib.parse import quote_plus
from sqlalchemy import create_engine

# Replace with your actual server, database, username, and password
server = 'localhost'
database = 'KC_labor'
username = 'root'
password = '****'
driver = 'SQL Server' 


connection_string = f"mysql+pymysql://{username}:{quote_plus(password)}@{server}/{database}"

# Create the engine
engine = create_engine(connection_string)




Write to SQL tables for easy access

In [40]:
Unemployment['Average of employment'].to_sql('employment',con=engine, if_exists='append', index=True)
Unemployment['Average of labor force'].to_sql('labor force',con=engine, if_exists='append', index=True)
Unemployment['Average of unemployment'].to_sql('unemployment',con=engine, if_exists='append', index=True)
Unemployment['Average of unemployment rate'].to_sql('unemployment rate',con=engine, if_exists='append', index=True)

for df in OWES:
    OWES[df].to_sql('OWES',con=engine, if_exists='append', index=True)

Sheet5.to_sql('CPI', con=engine, if_exists='append', index=True)

SQL queries

In [ ]:
import mysql.connector

conn = mysql.connector.connect(host=server,
                               user=username,
                               password=password,
                               database=database)


#Example
query = 'SELECT * FROM `OWES`'
df = pd.read_sql(query, conn, index_col='Year')
df.head()

EDA

Replacing missing values

In [ ]:
query = 'SELECT * FROM `OWES`'
OWES = pd.read_sql(query, conn, index_col='Year')
# indicated a wage equal to or greater than $115.00 per hour or $239,200 per year 
OWES['H_MEDIAN'] = OWES['H_MEDIAN'].replace('#', '>=115')
OWES['A_MEDIAN'] = OWES['A_MEDIAN'].replace('#', '239200')
OWES[OWES['OCC_TITLE']=='Physicians and Surgeons, All Other']

In [ ]:
OWES['A_MEDIAN'] = pd.to_numeric(OWES['A_MEDIAN'], errors='coerce')
OWES['TOT_EMP'] = pd.to_numeric(OWES['TOT_EMP'], errors='coerce')
OWES[OWES['A_MEDIAN'].isnull()].sort_values(['TOT_EMP'],ascending=False).head(30)

Are wages keeping up with inflation?

In [ ]:
#CPI yearly average
query = 'SELECT * FROM `CPI`'
CPI = pd.read_sql(query, conn, index_col='Year')
CPI = CPI.apply(lambda x: x.astype('float'))
CPI = CPI.mean(axis=1)
CPI = CPI / CPI.shift(1)
CPI.fillna(0, inplace=True)
CPI.drop(2024, inplace=True)
CPI.name = 'CPI'

#Salary pct increase year over year in major sectors
MajorSectors = OWES[OWES['O_GROUP']=='major']
MajorSectors.index = MajorSectors.index.astype('int64')
MajorSectors = MajorSectors.sort_values(['OCC_TITLE', 'Year'])
#MUST USE TRANSFORM INSTEAD OF APPLY TO MATCH INDEX LENGTH
MajorSectors['SalaryIncrease'] = MajorSectors.groupby('OCC_TITLE')['A_MEDIAN'].transform(lambda x: x / x.shift(1)).fillna(0)

In [ ]:
Salary_CPI = MajorSectors.merge(CPI, left_index=True, right_index=True, how='left')
Salary_CPI
Salary_CPI['Rate_over_CPI'] = Salary_CPI['SalaryIncrease'] - Salary_CPI['CPI']
Salary_CPI['Rate_over_CPI']

#Drop not complete data series
print(Salary_CPI.groupby('OCC_TITLE').size())
Salary_CPI = Salary_CPI[Salary_CPI['OCC_TITLE'] != 'Educational Instruction and Library Occupations']
Salary_CPI = Salary_CPI[Salary_CPI['OCC_TITLE'] != 'Education, Training, and Library Occupations']

In [ ]:
#COMPARE
SalaryIncrease_over_CPIIncrease = Salary_CPI.groupby('OCC_TITLE')['Rate_over_CPI'].mean().sort_values()
SalaryIncrease_over_CPIIncrease.index = SalaryIncrease_over_CPIIncrease.index.str.replace(' Occupations', '')

print(SalaryIncrease_over_CPIIncrease)

Major sectors for which average salary increase is not keeping up with inflation


In [ ]:
DecreasingSalary = SalaryIncrease_over_CPIIncrease[SalaryIncrease_over_CPIIncrease<0]
DecreasingSalary = DecreasingSalary.reset_index()
DecreasingSalary.columns = ['Category', 'Value']

with sns.axes_style('white'):
    g = sns.catplot(data = DecreasingSalary, x = "Category", y="Value", aspect = 1.5,
                       kind = "bar", palette='Set2')
    

plt.gca().invert_yaxis()
g.set_xticklabels(rotation=45, horizontalalignment='right')    
g.set_axis_labels("Category", "Change after Inflation")
plt.title('Sectors with Decreasing Salary', y=1, fontsize = 12)

plt.show()

Major sectors for which average salary increase is greatest over inflation


In [ ]:
IncreasingSalary = SalaryIncrease_over_CPIIncrease.sort_values(ascending=False).head(5)

IncreasingSalary = IncreasingSalary.reset_index()
IncreasingSalary.columns = ['Category', 'Value']

with sns.axes_style('white'):
    g = sns.catplot(data = IncreasingSalary, x = "Category", y="Value", aspect = 1,
                       kind = "bar", palette = 'Set2')
    
plt.subplots_adjust(left=0.45, right=1.25, top=0.9, bottom=0.3)
g.set_xticklabels(rotation=65, horizontalalignment='right')    
g.set_axis_labels("", "Change after Inflation")
plt.title('Sectors with Greatest Salary Increase', y=1, fontsize = 12)
g.fig.set_size_inches(5, 5.1)

plt.show()

Sectors with Highest Employment Growth

In [ ]:

MajorSectors['NewJobs'] = MajorSectors.groupby('OCC_TITLE')['TOT_EMP'].transform(lambda x: x - x.shift(1)).fillna(0)
#Number of new jobs over last 10 years
KCEmploymentGrowth = MajorSectors.groupby('OCC_TITLE')['NewJobs'].sum().sort_values(ascending=False)
KCEmploymentGrowth


In [ ]:
# KCEmploymentGrowth.head(5).plot(kind='bar')
KCEmploymentGrowth = MajorSectors.groupby('OCC_TITLE')['NewJobs'].sum().sort_values(ascending=False)
KCEmploymentGrowthTop = KCEmploymentGrowth.head(5)
KCEmploymentGrowthTop.index = KCEmploymentGrowthTop.index.str.replace(' Occupations', '')
KCEmploymentGrowthTop

KCEmploymentGrowthTop = KCEmploymentGrowthTop.reset_index()
KCEmploymentGrowthTop.columns = ['Category', 'Value']
KCEmploymentGrowthTop

with sns.axes_style('white'):
    g = sns.catplot(data = KCEmploymentGrowthTop, x = "Category", y="Value", aspect = 1,
                       kind = "bar", palette = 'Set2')
    
g.set_xticklabels(rotation=65, horizontalalignment='right')    
g.set_axis_labels("", "New Jobs")
plt.title('Sectors with Most Job Growth', y=1, fontsize = 12)
plt.subplots_adjust(left=0.55, right=1.25, top=0.9, bottom=0.3)
g.fig.set_size_inches(5.05, 5)


plt.show()

Emphasis on Transportation and Material Moving Occupations

In [ ]:
Transportation = OWES[OWES['OCC_CODE'].str.startswith('53')]
Transportation = Transportation[Transportation['OCC_TITLE'] != 'Transportation and Material Moving Occupations']
#Some data was not available for all years and cannot be included
Transportation = Transportation[Transportation['OCC_TITLE'].map(Transportation['OCC_TITLE'].value_counts()) == 10]
Transportation['NewJobs'] = Transportation.groupby('OCC_TITLE')['TOT_EMP'].transform(lambda x: x - x.shift(1)).fillna(0)
Transportation.groupby('OCC_TITLE')['NewJobs'].sum().sort_values(ascending=False)

Job Titles with most new jobs overall

In [ ]:
Detailed = OWES[OWES['O_GROUP']=='detailed']
Detailed.index = Detailed.index.astype('int64')
Detailed = Detailed[Detailed['OCC_TITLE'].map(Detailed['OCC_TITLE'].value_counts()) == 10]
Detailed['NewJobs'] = Detailed.groupby('OCC_TITLE')['TOT_EMP'].transform(lambda x: x - x.shift(1)).fillna(0)

#Number of new jobs over last 10 years
JobGrowth = Detailed.groupby('OCC_TITLE')['NewJobs'].sum().sort_values(ascending=False)
JobGrowth = JobGrowth.head(5)

JobGrowth = JobGrowth.reset_index()
JobGrowth.columns = ['Category', 'Value']

with sns.axes_style('white'):
    g = sns.catplot(data = JobGrowth, x = "Category", y="Value", aspect = 1,
                       kind = "bar", palette = 'Set2')
    
plt.subplots_adjust(left=0.55, right=1.25, top=0.9, bottom=0.3)
g.set_xticklabels(rotation=65, horizontalalignment='right')    
g.set_axis_labels("Category", "New Jobs")
plt.title('Jobs with Most Growth Overall', y=1, fontsize = 12)

plt.show()

Kansas City VS U.S. Employment Growth by Sector over Decade

In [ ]:
OWESNational = pd.concat(OWESN.values())

OWESNmajor = OWESNational[OWESNational['OCC_GROUP'] == 'major']
OWESNmajor['NewJobs'] = OWESNmajor.groupby('OCC_TITLE')['TOT_EMP'].transform(lambda x: x - x.shift(1)).fillna(0)
NEmploymentGrowth = OWESNmajor.groupby('OCC_TITLE')['NewJobs'].sum().sort_values(ascending=False)

KCStarting_Employment = MajorSectors[MajorSectors.index == 2014]
KCStarting_Employment['OCC_TITLE'] = KCStarting_Employment['OCC_TITLE']
KCStarting_Employment['EMP_GROWTH'] = KCStarting_Employment['OCC_TITLE'].map(KCEmploymentGrowth)

KCStarting_Employment['PCT_CHANGE'] = KCStarting_Employment['EMP_GROWTH'] / KCStarting_Employment['TOT_EMP']
KCEmpCHANGE = KCStarting_Employment[['OCC_TITLE', 'PCT_CHANGE']]
KCEmpCHANGE.set_index('OCC_TITLE', inplace=True)

NStarting_Employment = OWESNmajor[OWESNmajor['Year'] == '2014']
NStarting_Employment['EMP_GROWTH'] = NStarting_Employment.index.map(NEmploymentGrowth)
NStarting_Employment['PCT_CHANGE'] = NStarting_Employment['EMP_GROWTH'] / NStarting_Employment['TOT_EMP']
NEmpCHANGE = NStarting_Employment[['PCT_CHANGE']]

#KC Percent Diffrence of Increase over National Average
#Positive Result: KC outperformed the nation, Negative Result: KC underperformed the nation.
percentage_difference = ((KCEmpCHANGE - NEmpCHANGE) / abs(NEmpCHANGE)) * 100
percentage_difference = percentage_difference.sort_values(by='PCT_CHANGE')
percentage_difference.index = percentage_difference.index.str.replace(' Occupations', '')

percentage_difference = percentage_difference.reset_index()
percentage_difference.columns = ['Category', 'Value']

g.fig.set_size_inches(20, 8)
with sns.axes_style('white'):
    g = sns.catplot(data = percentage_difference, y = "Category", x="Value", aspect = 1.25,
                       kind = "bar", palette = 'Set2', height=6)
    
g.fig.set_size_inches(10, 5)
    
plt.subplots_adjust(left=0.35, right=.8, top=.9, bottom=0.3)
g.set_xticklabels(horizontalalignment='center')    
g.set_axis_labels("Percent of National Growth", '')
plt.title('Job Growth Relative to National', y=1, fontsize = 12)
g.ax.axvline(0, color='black', linewidth=1, linestyle='--')

plt.show()

Kansas City Average Unemployment Rate 2014-2023

In [ ]:
queryEmpl = 'SELECT `Year`, `Grand Total` FROM `unemployment rate` WHERE year BETWEEN 2014 AND 2023'
Avg_Unemp_Rate = pd.read_sql(queryEmpl, conn, index_col='Year')
Avg_Unemp_Rate.columns = ['Avg Unemp Rate %']
Avg_Unemp_Rate = Avg_Unemp_Rate.astype('float').round(2)
Avg_Unemp_Rate.plot(kind='line', title="Kansas City Unemployment Rate", fontsize=12)